<a href="https://colab.research.google.com/github/songqsh/foo1/blob/master/src/Simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplex method

We are going to illustrate simplex tableau method for linear programing using the following example - WG:
$$\max z = 3 x_1 + 5 x_2$$
s.t.
$$\begin{array}{lll}
x_1 & & \le 4 \\
& 2 x_2 &\le 12 \\
3x_1 & + 2x_2 & \le 18\\
x_1, & x_2 & \ge 0
\end{array}
$$

In [1]:
# import package
import numpy as np
import numpy.linalg as la
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd

## Matrix in numpy

## Pivotizing

Pivotizing with $(i,j)$-entry means converting the matrix by multiple EROS so that the resulting matrix has all zeros in $j$-column, but one in $(i,j)$-entry. This is the key step in updating basic feasible solution.

In [2]:
#pivotize with (i,j)
def pivot(A, i, j):
  A[i] = A[i]/A[i,j] #scale to get one in (i,j)
  n_rows, _ = A.shape
  for k in range(n_rows):
    if k==i:
      continue # skip i-row
    A[k] = A[k] - A[i]*A[k,j] # replacement to get zero

## Simplex tableau

In [12]:
def newpivot(M): #M is the tableau
  optimal_test = min(M[0, range(1,6)])
  m, n = M.shape
  m = m-1 #number of constraints in the augmented form
  n = n-2 #number of variables in the augmented form
  if optimal_test<0:
    pivot_col = np.argmin(M[0,range(1,n+1)])+1
  else:
    print(f'pass the optimal test')
    return(0)
  ratio_list = np.divide(M[range(1,m+1),-1], M[range(1, m+1),pivot_col])
  ratio_list[ratio_list<0]=0
  if np.min(ratio_list)==np.inf or np.min(ratio_list)<=0:
    print(f'no leaving variable, here is the ratio list {ratio_list}')
    return(0)
  else:
    pivot_row = np.argmin(ratio_list)+1
  return(pivot_row, pivot_col)

In [13]:
# input initial tableau
M = np.array([
               [1, -3, -5, 0, 0, 0, 0], 
               [0, 1, 0, 1, 0, 0, 4], 
               [0, 0, 2, 0, 1, 0, 12],
               [0, 3, 2, 0, 0, 1, 18]
               ], dtype = float)
m, n = M.shape
m = m-1
n = n-2
print(f'{m} constraints and {n} variables')

pd.DataFrame(M) # print matrix

3 constraints and 5 variables


,0,1,2,3,4,5,6
0,1.0,-3.0,-5.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,4.0
2,0.0,0.0,2.0,0.0,1.0,0.0,12.0
3,0.0,3.0,2.0,0.0,0.0,1.0,18.0


In [16]:
npivot = newpivot(M)
while npivot != 0:
  print(f'new pivot is {npivot}')
  if npivot is not 0:
    pivot(M, npivot[0], npivot[1])
    print(pd.DataFrame(M))
    print(f'=======================')
    npivot = newpivot(M)

new pivot is (2, 2)
     0    1    2    3    4    5     6
0  1.0 -3.0  0.0  0.0  2.5  0.0  30.0
1  0.0  1.0  0.0  1.0  0.0  0.0   4.0
2  0.0  0.0  1.0  0.0  0.5  0.0   6.0
3  0.0  3.0  0.0  0.0 -1.0  1.0   6.0
new pivot is (3, 1)
     0    1    2    3         4         5     6
0  1.0  0.0  0.0  0.0  1.500000  1.000000  36.0
1  0.0  0.0  0.0  1.0  0.333333 -0.333333   2.0
2  0.0  0.0  1.0  0.0  0.500000  0.000000   6.0
3  0.0  1.0  0.0  0.0 -0.333333  0.333333   2.0
pass the optimal test


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Since the 0-row has no negative number, the last entry of the first row is the maximum value, which is 36.